In [16]:
import json
import pandas as pd
import numpy as np
import re
import json

In [17]:
# read in the original json file
'''!!!!!!!!!!!!! Note the file position and file name !!!!!!!!!!!!!!'''
path = "postcode3000.json"
with open(path, "r") as f:
    property_data = json.load(f)

In [18]:
property_df = pd.DataFrame()

rent = []
address =[]
features = []
type = []
furnitured = [] 
pool = []
gym = []
latitude = []
longitude = []


for i in property_data:
    address.append(property_data[i]["name"])
    rent.append(property_data[i]["cost_text"])
    features. append(property_data[i]["features"])
    type.append(property_data[i]["type"])
    furnitured.append(property_data[i]["furnitured"])
    pool.append(property_data[i]["pool"])
    gym.append(property_data[i]["gym"])
    latitude.append(property_data[i]["coordinates"][0])
    longitude.append(property_data[i]["coordinates"][1])


property_df["address"] = address
property_df["rent"] = rent
property_df["features"] = features
property_df["type"] = type
property_df["furnitured"] = furnitured
property_df["pool"] = pool
property_df["gym"] = gym
property_df["latitude"] = latitude
property_df["longitude"] = longitude

property_df.head()

,address,rent,features,type,furnitured,pool,gym,latitude,longitude
0,2108/288 Spencer Street Melbourne VIC 3000,$350,1 Bed1 Bath− Parking,Apartment / Unit / Flat,Yes,Yes,No,-37.813775,144.952095
1,1505/250 Elizabeth Street Melbourne VIC 3000,$520.00 per week,2 Beds1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.812564,144.962818
2,2413/220 Spencer Street Melbourne VIC 3000,$450 per week,2 Beds2 Baths1 Parking,Apartment / Unit / Flat,No,Yes,No,-37.815716,144.952956
3,2205/22-24 Jane Bell Lane Melbourne VIC 3000,$550,2 Beds1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.810220,144.966265
4,G11/61 MacKenzie Street Melbourne VIC 3000,$550.00 per week,2 Beds1 Bath1 Parking,Apartment / Unit / Flat,Yes,No,No,-37.807871,144.966303


In [19]:
property_df.shape

(15459, 9)

In [20]:
unique_property_df = property_df.drop_duplicates(keep="first").reset_index(drop = True)
unique_property_df.shape

(15345, 9)

In [21]:
# get postcode for all property
postcode=[]
for i in range(len(unique_property_df)):
    postcode.append(int(unique_property_df["address"][i][-4:]))

print("The number of postcode is: ", len(postcode))
unique_property_df["postcode"] = postcode

The number of postcode is:  15345


In [22]:
unique_property_df.shape

(15345, 10)

In [23]:
clean_features = unique_property_df[unique_property_df["features"] != ""].reset_index(drop=True)
clean_features.shape

(15313, 10)

In [24]:
f = clean_features["features"]
f.to_csv("fea.csv")

In [25]:
f[12256]

'3 Beds1 Bath1 Parking'

In [26]:
(re.findall(r'(\d|-|−)\s?Park', "- Beds1 Bath− Parking"))[0].isdigit()

False

In [27]:
(re.findall(r'(\d|-|−)\s?Bed', "- 2 Beds1 Bath− Parking"))

['2']

In [28]:
# "- 2 Beds1 Bath− Parking".extract(r'(\d|-|−)\s?Bed', expand = True)

In [29]:
clean_features = unique_property_df[unique_property_df["features"] != ""].reset_index(drop=True)

num_bed_list = []
num_bath_list = []
num_park_list = []

for i in clean_features["features"]:

    # find the number of bedroom, bathroom and parking area
    num_bed = re.findall(r'(\d|-|−)\s?Bed', i)
    num_bath = re.findall(r'(\d|-|−)\s?Bath', i)
    num_park = re.findall(r'(\d|-|−)\s?Park', i)

    # if no information, set the number to zero
    if (len(num_bed) == 0) or (num_bed[0].isdigit() == False):
        num_bed = ["0"]
    if (len(num_bath) == 0) or (num_bath[0].isdigit() == False):
        num_bath = ["0"]
    if (len(num_park) == 0) or (num_park[0].isdigit() == False):
        num_park = ["0"]
    
    # append them together
    num_bed_list.append(int(num_bed[0]))
    num_bath_list.append(int(num_bath[0]))
    num_park_list.append(int(num_park[0]))

# generate the new features of the property
clean_features["num_bed"] = num_bed_list
clean_features["num_bath"] = num_bath_list
clean_features["num_park"] = num_park_list

In [30]:
clean_features.head()

,address,rent,features,type,furnitured,pool,gym,latitude,longitude,postcode,num_bed,num_bath,num_park
0,2108/288 Spencer Street Melbourne VIC 3000,$350,1 Bed1 Bath− Parking,Apartment / Unit / Flat,Yes,Yes,No,-37.813775,144.952095,3000,1,1,0
1,1505/250 Elizabeth Street Melbourne VIC 3000,$520.00 per week,2 Beds1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.812564,144.962818,3000,2,1,0
2,2413/220 Spencer Street Melbourne VIC 3000,$450 per week,2 Beds2 Baths1 Parking,Apartment / Unit / Flat,No,Yes,No,-37.815716,144.952956,3000,2,2,1
3,2205/22-24 Jane Bell Lane Melbourne VIC 3000,$550,2 Beds1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.810220,144.966265,3000,2,1,0
4,G11/61 MacKenzie Street Melbourne VIC 3000,$550.00 per week,2 Beds1 Bath1 Parking,Apartment / Unit / Flat,Yes,No,No,-37.807871,144.966303,3000,2,1,1
